<a href="https://colab.research.google.com/github/heghiw/DeepLearningPetIdentification/blob/main/fine11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:

# Correct raw URL for the utility.py file
url = "https://raw.githubusercontent.com/avkaz/DeepLearningPetIdentification/main/utility.py"

# Fetch and save the file locally
response = requests.get(url)
with open("utility.py", "wb") as f:
    f.write(response.content)


from utility import download_and_preprocess_image


In [1]:
## 1st -  Download utility.py file from github repository
## 2nd - Imports all functions from utility.py

import requests


import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from io import BytesIO
import pandas as pd
from google.colab import files
!pip install faiss-cpu
from tensorflow.keras.applications import EfficientNetB0
import faiss
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas

utility.py downloaded successfully.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.7 MB/s eta 0:00:00


In [2]:
model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
data = utility.get_data()

In [4]:
filtered_data = {k: v for k, v in data.items() if len(v['images']) > 3}

In [80]:
from itertools import islice
first_100 = dict(islice(filtered_data.items(), 100))

filter out pets for training, only using postings that have 3+ pictures for triplet loss

In [51]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.losses import Loss
from sklearn.model_selection import train_test_split
import faiss
import requests
from PIL import Image
import io

# Assuming utility.py with download_and_preprocess_image is imported
# from utility import download_and_preprocess_image


def filter_pets_by_images(data, min_images=3):
    """Filters pets with at least 'min_images' images."""
    return {key: value for key, value in data.items() if len(value['images']) >= min_images}


def triplet_loss(margin=1.0):
    """Triplet Loss function."""
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
        positive_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        negative_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        loss_value = tf.maximum(positive_distance - negative_distance + margin, 0.0)
        return tf.reduce_mean(loss_value)
    return loss

from tensorflow.keras import layers, Model
import tensorflow as tf

def create_model(input_dim):
    """Create a model that processes embeddings directly (e.g., from EfficientNetB0)."""
    inputs = tf.keras.Input(shape=(input_dim,))  # Input layer for the flattened embedding vector
    x = layers.Dense(256, activation='relu')(inputs)  # Dense layer to process embeddings
    x = layers.Dense(128, activation='relu')(x)  # Another dense layer for more abstraction
    output = layers.Dense(1, activation='sigmoid')(x)  # Output layer (adjust if needed)
    model = Model(inputs, output)
    return model

def generate_embeddings(model, image_urls):
    """Generate embeddings for the list of image URLs."""
    embeddings = []
    for url in image_urls:
        img = utility.download_and_preprocess_image(url, target_size=(128, 128))
        emb = model.predict(np.expand_dims(img, axis=0))
        embeddings.append(emb.flatten())
    return np.array(embeddings)

retry

In [45]:
# Create a mapping of pet identifiers to integer indices
pet_ids = list(first_100.keys())
pet_to_idx = {pet_id: idx for idx, pet_id in enumerate(pet_ids)}

# Prepare the data for triplet training
triplets = []
labels = []

for pet_key, pet_info in first_100.items():
    # Use the first 3 images for each pet as an example
    images = pet_info['images']
    embeddings = generate_embeddings(model, images)

    # Create triplets: anchor, positive, and negative
    for i in range(len(embeddings)):
        anchor = embeddings[i]
        positive = embeddings[(i + 1) % len(embeddings)]  # Next image (positive sample)

        # Pick a random negative example (different pet)
        negative_pet_key = np.random.choice([key for key in test_data if key != pet_key])
        negative_embedding = generate_embeddings(model, test_data[negative_pet_key]['images'])[0]

        # Add the triplet to the list
        triplets.append([anchor, positive, negative_embedding])
        labels.append(pet_to_idx[pet_key])  # You can adjust this if you want to use labels for something else

# Convert triplets to numpy array
triplets = np.array(triplets)

# Flatten the embeddings for FAISS
flattened_triplets = []

for triplet in triplets:
    anchor, positive, negative = triplet
    flattened_triplets.append(np.concatenate([anchor.flatten(), positive.flatten(), negative.flatten()]))

flattened_triplets = np.array(flattened_triplets)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━

In [47]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(flattened_triplets, labels, test_size=0.2, random_state=42)

# Initialize FAISS index for similarity search
embedding_dim = X_train.shape[1]  # The size of the flattened embeddings
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric for similarity search

# Add training data embeddings to the FAISS index
index.add(X_train)  # No need to reshape, as X_train is already flat

# Build the model (assuming create_model and triplet_loss are defined elsewhere)
input_dim = 3840

# Create the model
model = create_model(input_dim)

# Compile and train the model
model.compile(optimizer='adam', loss=triplet_loss(margin=1.0))
model.fit(X_train, np.array(y_train), epochs=10, batch_size=32)


In [63]:
query = X_test[0].reshape(1, -1)  # Example query: the first test sample (flattened)
D, I = index.search(query, k=5)  # Search for the 5 nearest neighbors
print("Nearest neighbors (indices):", I)
print("Distances:", D)

Nearest neighbors (indices): [[ 57  86  97 123  60]]
Distances: [[0.00041626 0.0007329  0.00079629 0.0010764  0.00107702]]


In [75]:
I_flat = [item for sublist in I for item in sublist]

# Filter the dictionary
filtered_pet_to_idx = {key: value for key, value in pet_to_idx.items() if value in I_flat}

print(filtered_pet_to_idx)

{}
